In [1]:
#匯入需要的套件
import matplotlib.pyplot as plt
from sklearn.utils import check_random_state
from gplearn.genetic import SymbolicRegressor
import pandas as pd

In [3]:
data =  pd.read_csv("E:/Lung_Cancer/Lung_Nodule_Segmentation/lung.csv")
label =  pd.read_csv("E:/Lung_Cancer/Lung_Nodule_Segmentation/labels.csv")['Std']

data.head()

,Start,Average
0,-698,-703.123
1,-699,-649.955
2,-644,-603.291
3,-625,-707.600
4,-293,-340.160


In [4]:
# 這邊呼叫gplearn 裡面 SymbolicRegressor的方法，該方法會用GP的方式去算符號回歸
# for i in range(20,21):
    # print("------"+str(i))
est_gp = SymbolicRegressor(
                        population_size=10000,       # population_size 族群大小(每一代個體數目)
                        generations=520,             # generations 演化次數(訓練次數)
                        stopping_criteria=24.55,     # stopping_criteria 停止演化的標準
                        p_crossover=0.7,             # p_crossover 交叉的機率
                        p_subtree_mutation=0.05,     # p_subtree_mutation 子節點突變機率
                        p_hoist_mutation=0.05,       # p_hoist_mutation 子節點提升突變機率
                        p_point_mutation=0.2,        # p_point_mutation 子節點點突變機率
                        max_samples=1,               # max_samples 訓練/驗證資料比例
                        verbose=1,                   # 顯示演化過程
                        parsimony_coefficient=0.01,  # 懲罰過於複雜的公式 0.01
                        random_state=0              
                        )
#開始訓練
est_gp.fit(data, label)


y_gp = est_gp.predict(data)                             # Return Predict Value
score_gp = est_gp.score(data, label)                    # Return the coefficient of determination of the prediction
print("Genetic Programming:",round(score_gp,3))         # Genetic Programming 演算法的分數

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    38.62      1.08582e+38       15          71.0003              N/A     77.42m
   1    18.19      1.00649e+12       31          66.4116              N/A     60.98m
   2    17.47       3.1947e+11       11          63.5165              N/A    111.65m
   3    13.40      1.55551e+06       25          49.9714              N/A     44.24m
   4    26.58       8.5483e+09       31          49.4901              N/A     48.89m
   5    49.89      4.20851e+07       21          42.3205              N/A     73.17m
   6    44.42      2.72364e+07       31          41.2904              N/A     75.75m
   7    34.02      1.64979e+07       89          40.5079              N/A     77.97m
   8    30.34      1.45796e+08       35          39.2316              N/A  

In [5]:
# predict
y_gp = est_gp.predict(data)                             # Return Predict Value
score_gp = est_gp.score(data, label)      # Return the coefficient of determination of the prediction
print(y_gp)
print("Genetic Programming:",score_gp)                                # Genetic Programming 演算法的分數

[ 53.45748572  58.95154191  67.00430718 ... 191.19216158 135.87162852
  47.34209233]
Genetic Programming: 0.7264717609679062


In [6]:
diff = 0
for i in range(len(y_gp)):
    diff = diff + abs(label[i] - y_gp[i])

print("Mean Error:", diff / len(y_gp))

Mean Error: 25.837894316107214


In [7]:
from sympy import sympify

converter = {
    'sub': lambda x, y : x - y,
    'div': lambda x, y : x/y,
    'mul': lambda x, y : x*y,
    'add': lambda x, y : x + y,
    'neg': lambda x    : -x,
    'pow': lambda x, y : x**y,
    'sin': lambda x    : sin(x),
    'cos': lambda x    : cos(x),
    'inv': lambda x: 1/x,
    'sqrt': lambda x: x**0.5,
    'pow3': lambda x: x**3
}

print(est_gp._program)
next_e = sympify(str(est_gp._program), locals=converter)
print(next_e)

add(div(0.736, 0.020), add(div(mul(mul(X1, -0.869), add(add(div(add(X1, sub(mul(mul(-0.384, X0), div(div(add(div(div(sub(X0, X1), div(-0.019, sub(add(mul(0.934, add(0.934, add(div(mul(X1, X1), sub(X1, X1)), sub(X0, X1)))), div(0.736, -0.029)), sub(X0, X1)))), div(-0.392, X1)), sub(X1, add(sub(X0, X1), div(0.870, div(0.491, add(div(0.736, -0.029), sub(X0, X1))))))), div(0.736, -0.029)), div(0.736, 0.025))), X0)), sub(div(-0.418, 0.316), div(add(mul(-0.384, X0), X0), div(-0.019, sub(add(mul(0.934, add(0.934, add(div(div(-0.359, X1), div(-0.029, X1)), sub(X0, X1)))), div(0.736, -0.029)), sub(X0, X1)))))), div(0.736, 0.020)), div(0.736, 0.020))), div(X1, -0.720)), div(0.736, 0.020)))
119.650048 + 0.62568*(-0.384*X0*(0.00370987272921086*X0 + 0.00341425780496509*X1*(X0 - X1)*(3.47368421052631*X0 + zoo*X1**2 - 3.47368421052631*X1 + 1289.83970235935) - 0.00504826178875718*X1 - 0.0601866200301072) - X0 + X1)/(-0.616*X0*(3.47368421052631*X0 - 3.47368421052631*X1 + 681.298867513612) - 1.322784810

#  Saving Model

In [8]:
import pickle
f = open('E:/Lung_Cancer/Lung_Nodule_Segmentation/GP/test1.txt','wb')
f.write(pickle.dumps(est_gp))
f.close()

#  Loading Model

In [9]:
import pickle
f = open('E:/Lung_Cancer/Lung_Nodule_Segmentation/GP/test1.txt','rb')
gp = pickle.loads(f.read())

# Testing

In [10]:
y_gp = gp.predict(data)                             # Return Predict Value
score_gp = gp.score(data, label)                    # Return the coefficient of determination of the prediction
print(y_gp)
print("Genetic Programming:", score_gp)              # Genetic Programming 演算法的分數

[ 53.45748572  58.95154191  67.00430718 ... 191.19216158 135.87162852
  47.34209233]
Genetic Programming: 0.7264717609679062
